In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
df_notes = pd.read_csv('/scratch/kh2383/MIMIC/1.4/NOTEEVENTS.csv')

/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/ipython-6.2.1-py3.6.egg/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_adm = pd.read_csv('/scratch/kh2383/MIMIC/1.4/ADMISSIONS.csv')
df_icu = pd.read_csv('/scratch/kh2383/MIMIC/1.4/ICUSTAYS.csv')
df_mechvent_d2 = pd.read_csv('../Data/d2_mechvent_cohort08Oct19.csv')
df_mechvent_d7 = pd.read_csv('../Data/d7_mechvent_cohort27Sep19.csv')
df_mechvent_d14 = pd.read_csv('../Data/d14_mechvent_cohort27Sep19.csv')
df_mechvent_entire = pd.read_csv('../Data/entire_mechvent_cohort_starttimes15Oct19.csv')

# add the INTIME Column from ICUSTAYS table
df_mechvent_d2 = pd.merge(df_mechvent_d2, df_icu[['ICUSTAY_ID', 'INTIME']], on = ['ICUSTAY_ID'], how = 'inner')

In [4]:
def process(df):
    '''
    Input
        df: the initial cohort
        
    output
        df_less_n_r: first icu stay cohort with notes within 48 hour starting from first vent time, with the following labels:
            COHORT:
                0 for not prolonged, 1 for more than 7 days, 2 for more than 14 days. So for prolonged for more than 7 days, should sum 1 & 2 cohort
            LABEL:
                0 for not prolonged, 1 for more than 7 days (including more than 14 days)
            DEATH_90:
                0 for not dead, 1 for dead within 90 days of 48 hours (action time) after first vent time 
            DAYS_UNTIL_DEATH: 
                continuous variable of days of death from the 
            
            this table only returns notes that are in the respiratory, nurses, nurse/others category
       
       df_less_n: the above table with all the categories
       
       df_physician: the tables for admissions that have physician's only. It includes four categories: physician, respiratory, nurses, nurse/others category
       
       df_notes_cohort: the above table with no 48h restriction
       
    '''
    
    # only use the first ICU stays STARTING from the FIRST VENT TIME
    df = df.sort_values(['HADM_ID','INTIME_x']).groupby('HADM_ID', as_index=False).first()
    
    # drop all the repetitive ventilation events, so now this df has each row correspond to unique one admission's first icu stay
    df = df[['ICUSTAY_ID', 'HADM_ID', 'ADMITTIME','DISCHTIME','FIRST_VENT_STARTTIME', 'DOD']].drop_duplicates().reset_index(drop = True)
    
    df.DOD = pd.to_datetime(df.DOD, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    df.ADMITTIME = pd.to_datetime(df.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    df.DISCHTIME = pd.to_datetime(df.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    df.FIRST_VENT_STARTTIME = pd.to_datetime(df.FIRST_VENT_STARTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

    # cohort 0: not prolonged, 1: more than 7 days, 2: more than 14 days
    df = df.assign(COHORT = [1 if i else 0 for i in (df.HADM_ID.isin(df_mechvent_d7.HADM_ID.unique()))])
    df.loc[df.HADM_ID.isin(df_mechvent_d14.HADM_ID.unique()), 'COHORT'] = 2
    count = df.groupby('COHORT').HADM_ID.nunique().values.tolist()
    print('Not prolonged: {}, more than 7 days: {}, more than 14 days: {}'.format(count[0], count[1], count[2]))
    
    #to calculate the death label
    df = df.assign(DAYS_UNTIL_DEATH = ((df['DOD']-df['FIRST_VENT_STARTTIME']).dt.total_seconds()/(60*60*24)))
    df = df.assign(DEATH = [1 if i else 0 for i in (df.DAYS_UNTIL_DEATH < 7)])
    df = df.assign(DEATH_90 = [1 if i else 0 for i in (df.DAYS_UNTIL_DEATH < 92)])
    
    df_curated = df.drop_duplicates().reset_index(drop = True)
    print('n (number of admissions) is : {}'.format(len(df_curated.HADM_ID.unique())))
    
    # filter so that notes only in the admission cohort, in the CATEGORY we want
    df_notes_cohort = df_notes[df_notes.HADM_ID.isin(df_curated.HADM_ID)]
    print('removed subjects with no notes associated, n: {}'.format(len(df_notes_cohort.HADM_ID.unique())))

    df_notes_cohort = pd.merge(df_notes_cohort[['HADM_ID', 'CHARTTIME', 'TEXT', 'CATEGORY', 'DESCRIPTION']], df_curated, on = ['HADM_ID'], how = 'inner')
    print('after merge, n: {}'.format(len(df_notes_cohort.HADM_ID.unique())))
    
    df_notes_cohort.CHARTTIME = pd.to_datetime(df_notes_cohort.CHARTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    df_notes_cohort.ADMITTIME = pd.to_datetime(df_notes_cohort.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
        
    TIME_to_VENT = ((df_notes_cohort['CHARTTIME']-df_notes_cohort['FIRST_VENT_STARTTIME']).dt.total_seconds()/(60*60))
    df_less_n = df_notes_cohort[ (TIME_to_VENT<=48) & (TIME_to_VENT >=0)]
    print('restricted to the first 48h, n: {}'.format(len(df_less_n.HADM_ID.unique())))
    df_less_n = df_less_n.assign(Label = [1 if i else 0 for i in (df_less_n.COHORT != 0)])

    # create cohort that have physician notes aka 2008-2013 MIMIC
    df_physician = df_less_n[df_less_n.CATEGORY.isin(['Physician ','Nursing', 'Nursing/other', 'Respiratory '])]
    df_physician = df_physician[df_physician.HADM_ID.isin(df_physician[df_physician.CATEGORY == 'Physician '].HADM_ID.unique())]
    
    print('There are {} # of admissions that have physician notes'.format(len(df_physician.HADM_ID.unique())))
    
    df_less_n_r = df_less_n[df_less_n.CATEGORY.isin(['Nursing', 'Nursing/other', 'Respiratory '])]

    print('remove subjects with no notes in the selected categories the entire admission, n: {}'.format(len(df_notes_cohort[df_notes_cohort.CATEGORY.isin(['Physician ', 'Nursing', 'Nursing/other'])].HADM_ID.unique())))
    print('remove subjects with no notes in the selected categories under 48h, n: {}'.format(len(df_less_n_r.HADM_ID.unique())))
    
    print('# of notes restricted to the three categories: {}'.format(len(df_less_n_r)))
    print('# of notes using all categories: {}'.format(len(df_less_n)))
    
    print('--- for prediction: below are label stats using three categories only ---')
    
    count = df_less_n_r.groupby('COHORT').HADM_ID.nunique().values.tolist()
    print('Not prolonged: {}, prolonged: {}, out of which, more than 7 days: {}, more than 14 days: {}'.format(count[0], count[1]+count[2], count[1], count[2]))
    count = df_less_n_r.groupby('DEATH').HADM_ID.nunique().values.tolist()
    print('Not Death within 7 days: {}, Death within 7 days: {}'.format(count[0], count[1]))
    
    df_death = df_less_n_r[df_less_n_r.HADM_ID.isin(df_less_n_r.groupby('DEATH').HADM_ID.unique()[1])]
    c = df_death[['HADM_ID','COHORT']].drop_duplicates().COHORT.value_counts().values
    try:
        print('Out of the {} death within 7 days, {} is from cohort not prolonged, {} is from cohort prolonged more than 7 days, and {} for 14 days'.format(count[1], c[0], c[1], c[2]))
    except:
        print('Out of the {} death within 7 days, {} is from cohort not prolonged, {} is from cohort prolonged more than 7 days, and 0 for 14 days'.format(count[1], c[0], c[1]))
        
    count = df_less_n_r.groupby('DEATH_90').HADM_ID.nunique().values.tolist()
    print('Not Death within 90 days: {}, Death within 90 days: {}'.format(count[0], count[1]))
        
    return df_less_n_r.reset_index(drop = True), df_less_n.reset_index(drop = True), df_physician.reset_index(drop = True), df_notes_cohort.reset_index(drop = True)
    

In [5]:
df_less_n_d2, df_less_48, df_physician, df_notes_d2 = process(df_mechvent_d2)

Not prolonged: 5185, more than 7 days: 2424, more than 14 days: 1874
n (number of admissions) is : 9483
removed subjects with no notes associated, n: 9372
after merge, n: 9372
restricted to the first 48h, n: 9095
There are 1538 # of admissions that have physician notes
remove subjects with no notes in the selected categories the entire admission, n: 7290
remove subjects with no notes in the selected categories under 48h, n: 7287
# of notes restricted to the three categories: 71439
# of notes using all categories: 120108
--- for prediction: below are label stats using three categories only ---
Not prolonged: 3875, prolonged: 3412, out of which, more than 7 days: 1858, more than 14 days: 1554
Not Death within 7 days: 6360, Death within 7 days: 927
Out of the 927 death within 7 days, 885 is from cohort not prolonged, 42 is from cohort prolonged more than 7 days, and 0 for 14 days
Not Death within 90 days: 4607, Death within 90 days: 2680


In [6]:
import re
import string

def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('--|__|==','',y)
    
    # remove punctuation, digits, spaces
    #y = y.translate(str.maketrans("", "", string.punctuation))
    y = y.translate(str.maketrans("", "", string.digits))
    y = " ".join(y.split())
    return y

def preprocessing(df_less_n, max_seq_len): 
    df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\r',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT']=df_less_n['TEXT'].str.lower()

    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: preprocess1(x))
    df_less_n['# of tokens'] = df_less_n['TEXT'].str.split().str.len()
    df_less_n = df_less_n[df_less_n['# of tokens'] > 5]
    
    df_concat = pd.DataFrame(df_less_n.groupby('HADM_ID')['TEXT'].apply(lambda x: "%s" % ' '.join(x))).reset_index()
    df_concat = df_concat.assign(Label = df_concat['HADM_ID'].apply(lambda x: df_less_n[df_less_n['HADM_ID']==x].Label.values[0]))
    df_concat = df_concat.assign(DEATH_90 = df_concat['HADM_ID'].apply(lambda x: df_less_n[df_less_n['HADM_ID']==x].DEATH_90.values[0]))

    df_less_n = df_concat
    df_len = len(df_less_n)

    want=pd.DataFrame({'HADM_ID':[],'TEXT':[],'Label':[],'DEATH_90':[]})
    for i in tqdm(range(df_len)):
        x=df_less_n.TEXT.iloc[i].split()
        n=int(len(x)/max_seq_len)
        for j in range(n):
            want=want.append({'TEXT':' '.join(x[j*max_seq_len:(j+1)*max_seq_len]),'Label':df_less_n.Label.iloc[i],'DEATH_90': df_less_n.DEATH_90.iloc[i], 'HADM_ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
        if len(x)%max_seq_len>100:
            want=want.append({'TEXT':' '.join(x[-(len(x)%max_seq_len):]),'Label':df_less_n.Label.iloc[i],'DEATH_90': df_less_n.DEATH_90.iloc[i],'HADM_ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
        
    return want

In [7]:
df_use_max_len = preprocessing(df_less_n_d2, 318)

100%|██████████| 7287/7287 [02:37<00:00, 46.26it/s]


In [19]:
path = DATAFOLDER_PATH
# create one untouchable test set first (instead of create five) just to enlarge the pretraining notes size
df_HADM_ID = df_use_max_len.HADM_ID.drop_duplicates().reset_index(drop = True)
test_id = df_HADM_ID.sample(frac = 0.1, replace = False, random_state = 1)
test = df_use_max_len[df_use_max_len.HADM_ID.isin(test_id.values)]
test.reset_index(drop = True).to_csv(path + '/test.csv')

train_val_id = df_HADM_ID[~df_HADM_ID.index.isin(test_id.index)]

for i in tqdm(range(1,6)):
    val_id = train_val_id.sample(frac = 1/9, replace = False, random_state = i)
    train_id = train_val_id[~train_val_id.index.isin(val_id.index)]
        
    train = df_use_max_len[df_use_max_len.HADM_ID.isin(train_id.values)]
    val = df_use_max_len[df_use_max_len.HADM_ID.isin(val_id.values)]
    train_val = df_use_max_len[df_use_max_len.HADM_ID.isin(train_val_id)]
    
    train.reset_index(drop = True).to_csv(path + '/train.csv')
    val.reset_index(drop = True).to_csv(path + '/val.csv')

100%|██████████| 5/5 [00:23<00:00,  4.63s/it]
